# 章 データラングリング：連結、結合、変形

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

# warningを非表示にする
import warnings
warnings.filterwarnings('ignore')

## 階層型インデックス

p243
- 階層型インデックスとは、インデックスに階層を持たせることができる機能。
- 階層型インデックスを使うと、データの部分集合を簡潔に抽出できる。

__シリーズの場合__

In [2]:
# 階層型インデックス
data = pd.Series(
    np.random.randn(9),
    index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
           [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [4]:
type(data.index)

pandas.core.indexes.multi.MultiIndex

__部分インデックス参照__

In [5]:
# 親インデックスを単一指定
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [6]:
# 親インデックスの範囲指定（その１）
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [7]:
# 親インデックスの範囲指定（その２）
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

__「内側」の階層（子インデックス）を指定して抽出__

In [8]:
# 親インデックスを全て取得、子インデックスが「２」の値を抽出
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

p244
- __`unstack`__メソッドを使えば、子インデックスを列ラベルにしたデータフレームに変形することが出来る。
- __`stack`__メソッドを使えば、列ラベルを行インデックスの子インデックスにしたデータフレームに変形することが出来る。


In [9]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [10]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

__（実験）年月日の階層インデックスの場合__

In [11]:
data2 = pd.Series(["a", "b", "c", "d", "e", "f"],
                  index=[['2018', '2018', '2018', '2017', '2016', '2016'],
                         [4, 4, 4, 5, 6, 6], [1, 2, 3, 1, 1, 3]])
data2

2018  4  1    a
         2    b
         3    c
2017  5  1    d
2016  6  1    e
         3    f
dtype: object

In [12]:
data2.unstack()

,,1,2,3
2016,6,e,NaN,f
2017,5,d,NaN,NaN
2018,4,a,b,c


__データフレームの場合__

p245
- 行、列の何れの方向にも階層型インデックスを作成できる。

In [13]:
# 行と列に階層型インデックスを持たせる
frame = pd.DataFrame(
    np.arange(12).reshape((4, 3)),
    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
    columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

p245
- __names__属性で、階層型インデックスに名前をつけられる。（軸の一部のみにインデックス名を付ける事はできない）

In [14]:
# 階層型インデックスに名前を付ける
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

p246
- シリーズと同様に列の部分インデックス参照が出来る。
- MultiIndexオブジェクトだけを作成して、再利用することも出来る。

In [15]:
# 部分インデックス参照
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [16]:
# 階層インデックスの手動作成
pd.MultiIndex.from_arrays(
    [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
    names=['state', 'color'])

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

### 階層の順序変更やソート

p246
- インデックス階層の順序を入れ替えるには、__`swaplevel`__メソッドを使う。
- 特定のインデックスでソートしたい場合は__`sort_inde`__メソッドを使う。

In [17]:
# 入れ替え前
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [18]:
# key1 と key2 を入れ替える。
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [19]:
# key2に対してソート（key1はlevel=0）
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [20]:
# key1とkey2を入れ替えた上で、一番外の行インデックスでソート
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 階層ごとの要約統計量

p246
- levelオプションを使うと、その軸での集計対象としたい階層を指定できる。（デフォルトは行:axis=0）

In [21]:
# key2に対する合計
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [22]:
# color列に対する合計
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

？__key1、key2もまとめて集計してみるには？__

### データフレームの列をインデックスに使う

p248
- __`set_index`__メソッドは、指定した1つ以上の列をインデックスとするデータフレームを新規作成する。
- __`drop=False`__で、インデックスに使用した列を残したまま、データフレームを作成できる。
- __reset_index__メソッドで__`set_index`__で作成したインデックスを列に戻す。

In [23]:
frame = pd.DataFrame({
    'a': range(7),
    'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'd': [0, 1, 2, 0, 1, 2, 3]
})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [24]:
# c、d列をインデックスとするデータフレームを作成
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [25]:
# インデックスに使用した列を残す。
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [26]:
# 行インデックスを列インデックスに戻す
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## データセットの結合とマージ

p249
- __`pandas.merge`__でデータフレームの行同士をキーで結合する。
- __`pandas.concat`__で縦や横方向に結合する。
- __`combine_first`__で重複するデータを持つ複数のオブジェクトをつなぎ合わせて、オブジェクトの欠損値を別のオブジェクトの値で穴埋めできる。

### データフレームをデータベース風に結合する
p249
- __`pandas.merge`__でデータフレームの行同士をキーで結合する。
- merge関数の引数は表8-2 を参照。(p254）

In [27]:
df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': range(7)
})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [28]:
display(df1, df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


p251
- デフォルトとして、どちらのデータフレームにも含まれる同じ列ラベルをキーとして用いる。

In [29]:
# df1とf2を列ラベルで内部結合する（片側にしか存在しない列ラベルの値は結合されない）
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


p251
- 結合キーは__`on`__で指定可能（推奨）。

In [30]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


p251
- キーとしたい列の名前が2つのデータフレームで異なっている場合、__`left_on`__、__`right_on`__ で、それぞれを個別に指定する。

In [31]:
df3 = pd.DataFrame({
    'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
    'data1': range(7)
})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [32]:
display(df3, df4)

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


In [33]:
# データフレーム毎に結合キーを指定
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


p251
- 結合の方法は__`how`__で指定できる（デフォルトはinner）。

In [34]:
# df1とdf2を外部結合（両データフレームのキーの和集合）
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


__例）左 外部結合__  
__`merge`__の第１引数のデータフレームに存在するキーで結合される。

In [35]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)}).sort_values(by="key") 

df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)}).sort_values(by="key") 

In [36]:
display(df1, df2)

,key,data1
2,a,2
4,a,4
0,b,0
1,b,1
5,b,5
3,c,3


,key,data2
0,a,0
2,a,2
1,b,1
3,b,3
4,d,4


In [37]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,a,2,0.0
1,a,2,2.0
2,a,4,0.0
3,a,4,2.0
4,b,0,1.0
5,b,0,3.0
6,b,1,1.0
7,b,1,3.0
8,b,5,1.0
9,b,5,3.0


__例）内部結合__  
両データフレームに存在するキーで結合される。

In [38]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,a,2,0
1,a,2,2
2,a,4,0
3,a,4,2
4,b,0,1
5,b,0,3
6,b,1,1
7,b,1,3
8,b,5,1
9,b,5,3


__例）外部結合__  
少なくとも何れかのデータフレームに存在するキーで結合される。

In [39]:
df1 = pd.DataFrame({
    'key': ['b', 'b', 'a', 'c', 'a', 'b'],
    'data1': range(6)
}).sort_values(by="key")

df2 = pd.DataFrame({
    'key': ['a', 'b', 'a', 'b', 'd'],
    'data2': range(5)
}).sort_values(by="key")

In [40]:
display(df1, df2)

,key,data1
2,a,2
4,a,4
0,b,0
1,b,1
5,b,5
3,c,3


,key,data2
0,a,0
2,a,2
1,b,1
3,b,3
4,d,4


In [41]:
pd.merge(df1, df2, on="key", how='outer')

,key,data1,data2
0,a,2.0,0.0
1,a,2.0,2.0
2,a,4.0,0.0
3,a,4.0,2.0
4,b,0.0,1.0
5,b,0.0,3.0
6,b,1.0,1.0
7,b,1.0,3.0
8,b,5.0,1.0
9,b,5.0,3.0


p253
- 複数のキーでマージを行いたい場合は、__`on`__に列名のリストを渡す。

In [42]:
left = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar'],
    'key2': ['one', 'two', 'one'],
    'lval': [1, 2, 3]
}).sort_values(by=["key1", "key2"])

right = pd.DataFrame({
    'key1': ['foo', 'foo', 'bar', 'bar'],
    'key2': ['one', 'one', 'one', 'two'],
    'rval': [4, 5, 6, 7]
}).sort_values(by=["key1", "key2"])

In [43]:
display(left, right)

,key1,key2,lval
2,bar,one,3
0,foo,one,1
1,foo,two,2


,key1,key2,rval
2,bar,one,6
3,bar,two,7
0,foo,one,4
1,foo,one,5


In [44]:
# key1、key2で結合
pd.merge(left, right, on=['key1', 'key2'])

,key1,key2,lval,rval
0,bar,one,3,6
1,foo,one,1,4
2,foo,one,1,5


p254
- __`suffixes`__オプションで、列名が重複した場合に、左のデータフレーム由来の列と右のデータフレーム由来の列のそれぞれの名前の末尾に加える文字列を指定できる。

In [45]:
# 列名key2 が重複
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,bar,one,3,one,6
1,bar,one,3,two,7
2,foo,one,1,one,4
3,foo,one,1,one,5
4,foo,two,2,one,4
5,foo,two,2,one,5


### インデックスによるマージ

p255
- 列ではなく行ラベルでマージしたい場合、__`left_index=True`__ または __`right_index=True`__（若しくは両オ
プション）を使用する。

In [46]:
left1 = pd.DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'data1': range(6)
})

right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [47]:
display(left1, right1)

,key,data1
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


__right1__の行インデックスをキーに__left1__の__key__列と内部結合する。

In [48]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,data1,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


__right1の行インデックスをキーに left1 の key列 と外部結合する。__

In [49]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,data1,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


__階層型インデックスを持ったデータのマージ__

In [50]:
lefth = pd.DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.)
})

righth = pd.DataFrame(
    np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
           [2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2'])

In [51]:
display(lefth, righth)

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

__lefthのkey1、key2列の値とrighthの複合インデックスと内部結合__

In [52]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


__lefthの key1、key2 列の値と righth の複合インデックスで外部結合__

In [53]:
pd.merge(
    lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [54]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [55]:
display(left2,right2)

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


p257
- 2つのデータフレームの結合キーをどちらもインデックスにすることが出来る。

In [56]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


p257
- インデックスによるマージを簡単にできるよう、__`joinメソッド`__が用意されている。
- __`join`__を用いる場合、データフレームには重複した名前の列があってはいけない。

In [57]:
# left2とright2をインデックスラベルで外部結合
# pd.merge(left2, right2, how='outer', left_index=True, right_index=True) の別表記
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [58]:
display(left1,right1)

,key,data1
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [59]:
# key列で左結合
left1.join(right1, on='key')

,key,data1,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


p258
- 汎用的なconcat関数を用いるのと同様の例

In [60]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [61]:
display(left2.join(right2), another)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [62]:
# left2.join(right2) 結果に対して another を join
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [63]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 軸に沿った連結

p259
- NumPyにはNumPy配列を連結するconcatenate関数がある。

In [64]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [65]:
# 列同士（1軸）を結合
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

__シリーズの場合__

p260
- pandasのconcat関数を呼び出すと、シリーズ内の値やインデックスが連結される。
- デフォルトでは __axis=0__ 方向に連結し、結果を新たなシリーズとして生成する。
- __axis=1__ を指定した場合は、データフレームを返し（axis=1 とは列方向を意味する）、行ラベルでソートされる。

In [66]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

display(s1, s2, s3)

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [67]:
# axis=0 で結合
pd.concat([s3, s2, s1])

f    5
g    6
c    2
d    3
e    4
a    0
b    1
dtype: int64

In [68]:
# データフレームを返却（各列が結合前のシリーズ、かつ行ラベルでソート）
pd.concat([s3, s2, s1], axis=1)

,0,1,2
a,NaN,NaN,0.0
b,NaN,NaN,1.0
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,5.0,NaN,NaN
g,6.0,NaN,NaN


p261
- __`join='inner'`__ を指定すると、共通したインデックスのみの集合（内部結合）が得られる。

In [69]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [70]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [71]:
# 共通したインデックスラベルでの内部結合
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


p261
- __`join_axes`__ オプションを用いると、連結後に残したいインデックスを指定する事が可能。

In [72]:
display(s1, s4)

a    0
b    1
dtype: int64

a    0
b    1
f    5
g    6
dtype: int64

In [73]:
# 結合後に残す行インデックスを指定
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


p262
- 単純に結合すると、結合前の状態が分からなくなる。
- 結合前の状態も分かるように結合するには複合インデックスを用いると分かりやすい。
- 複合インデックスを用いた結合は __keysオプション__ を使用する。　

In [74]:
display(s1, s2, s3)

a    0
b    1
dtype: int64

c    2
d    3
e    4
dtype: int64

f    5
g    6
dtype: int64

In [75]:
# keysオプションを使用しない場合
pd.concat([s1, s1, s3])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [76]:
# keysオプションを使用して複合インデックスを作成
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [77]:
# keysの要素数 < 結合オブジェクト数
pd.concat([s1, s1, s3], keys=['one', 'two'])

one  a    0
     b    1
two  a    0
     b    1
dtype: int64

In [78]:
# keysの要素数 > 結合オブジェクト数
pd.concat([s1, s1, s3], keys=['one', 'two', 'three', 'four'])

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

p262
- __`axis=1`__ を指定してシリーズを列方向に結合する場合は、__keys__に与えた値はデータフレームの行のヘッダになる。

In [79]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


__データフレームの場合__

p262
- シリーズと考え方は同じ。

p263
- ディクショナリを用いると、ディクショナリのキーが __keysオプション__ で渡したのと同じ効果となる。
- __`names`オプション__で、軸の各階層に名前を付けることが可能。

p264
- __`ignore_index=True`__を指定すればインデックスが振り直される。

In [80]:
df1 = pd.DataFrame(
    np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])

df2 = pd.DataFrame(
    5 + np.arange(4).reshape(2, 2),
    index=['a', 'c'],
    columns=['three', 'four'])

In [81]:
display(df1, df2)

,one,two
a,0,1
b,2,3
c,4,5


,three,four
a,5,6
c,7,8


In [82]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [83]:
# ディクショナリを使うとkeysと同じ効果
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [84]:
# 複合インデックスの各軸に名前を付けることが出来る。
pd.concat([df1, df2],
          axis=1,
          keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [85]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

display(df1, df2)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741


,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


In [86]:
pd.concat([df1, df2])

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
0,-1.021228,0.476985,NaN,3.248944
1,0.302614,-0.577087,NaN,0.124121


In [87]:
# インデックスが振り直される
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


### 重複のあるデータの結合

__インデックスが重複したデータの一部だけを結合する例__

In [88]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(
    np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])

b[-1] = np.nan
display(a, b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [89]:
# a の値が欠損値であれば、ｂの値で埋める。
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

p265
- シリーズの__`combine_firstメソッド`__ を使うと __`np.where`__と同様の事が出来る。  
（インデックスがソートされるはずだが、ソートされていない）

In [90]:
b.combine_first(a)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [91]:
df1 = pd.DataFrame({
    'a': [1., np.nan, 5., np.nan],
    'b': [np.nan, 2., np.nan, 6.],
    'c': range(2, 18, 4)
})

df2 = pd.DataFrame({
    'a': [5., 4., np.nan, 3., 7.],
    'b': [np.nan, 3., 4., 6., 8.]
})

display(df1, df2)

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [92]:
# df1の欠損地はdf2の値で埋める
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 変形とピボット操作

### 階層型インデックスによる変形

p267
- __`stack`__　　データ内の各列を行へとピボット（「回転」）させる。
- __`unstack`__　各行を列へと回転させる。

In [93]:
data = pd.DataFrame(
    np.arange(6).reshape((2, 3)),
    index=pd.Index(['Ohio', 'Colorado'], name='state'),
    columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [94]:
# 列を行へ入れ替え（元の行インデックスは変更なし）
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [95]:
# stackを元に戻す(行を列に入れ替える。resultは影響を受けない)
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


p268
- デフォルトでは、最も内側の階層が __`stack` `unstack`__の対象となる。
- 番号やラベル名を引数に渡して対象となる階層を指定する事が出来る。

In [96]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [97]:
# インデックス番号を指定して入れ替え。
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [98]:
result.unstack(1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [99]:
# ラベル名を指定して入れ替え
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


p268
- __`unstack`__する際に、対象となる階層の値のうち一部が含まれていないサブグループがある場合は、欠損値が埋め込まれる。
- __`stack`__する際、欠損値を埋め込むようにするには、__`dropna=False`__ オプションを使う。

In [100]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])

# 複合インデックスを持ったシリーズ
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [101]:
# 内側（a,b,c・・・）で入れ替え。欠損値が埋め込まれる。
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [102]:
# stackの デフォルトは欠損値が埋め込まれない。
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [103]:
# 欠損値を埋め込むように指定。
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

p269
- データフレームをunstackすると、unstack対象となった階層が、生成されたデータフレームの列の最も低い階層に入る。

In [104]:
df = pd.DataFrame({
    'left': result,
    'right': result + 5
}, columns=pd.Index(['left', 'right'], name='side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [105]:
# state列が最も低い列の階層に入る
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [106]:
# side列が最も低い行の階層に入る
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### 「縦持ち」フォーマットから「横持ち」フォーマットへのピボット

p270
- 複数の時系列データをデータベースやCSVファイルに保存する方法としては、long（縦持ち）フォーマットや積み上げ型フォーマットがよく使われる。

-273
- __`pivot`__ を呼び出すのは、__set_index__ を呼び出した後に __unstack__ を呼び出して階層型インデックスを作成するのと同じ。

In [107]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [108]:
# yearとquarterという2つの列を組み合わせて、期間を表す型を作成。PeriodInde については 11章
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
data.head()

# dateとitem をRDBにおける主キーにしたlong（縦持ち）フォーマット
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


__複合キー（date、item）を分解__

In [109]:
# date - itemの種類毎にvalueを表示。
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2
1960-06-30,0.14,2834.390,5.2
1960-09-30,2.70,2839.022,5.6
1960-12-31,1.21,2802.616,6.3
1961-03-31,-0.40,2819.264,6.8


In [110]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,0.523772
1,1959-03-31,infl,0.000,0.000940
2,1959-03-31,unemp,5.800,1.343810
3,1959-06-30,realgdp,2778.801,-0.713544
4,1959-06-30,infl,2.340,-0.831154
5,1959-06-30,unemp,5.100,-2.370232
6,1959-09-30,realgdp,2775.488,-1.860761
7,1959-09-30,infl,2.740,-0.860757
8,1959-09-30,unemp,5.300,0.560145
9,1959-12-31,realgdp,2785.204,-1.265934


In [111]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.000940  0.523772  1.343810
1959-06-30  2.34  2778.801   5.1 -0.831154 -0.713544 -2.370232
1959-09-30  2.74  2775.488   5.3 -0.860757 -1.860761  0.560145
1959-12-31  0.27  2785.204   5.6  0.119827 -1.265934 -1.063512
1960-03-31  2.31  2847.699   5.2 -2.359419  0.332883 -0.199543

In [112]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [113]:
# pivotを呼び出すのは、set_indexの後にunstackを呼び出して階層型インデックスを作成するのと同じ
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8  0.000940  0.523772  1.343810
1959-06-30  2.34  2778.801   5.1 -0.831154 -0.713544 -2.370232
1959-09-30  2.74  2775.488   5.3 -0.860757 -1.860761  0.560145
1959-12-31  0.27  2785.204   5.6  0.119827 -1.265934 -1.063512
1960-03-31  2.31  2847.699   5.2 -2.359419  0.332883 -0.199543

### 「横持ち」フォーマットから「縦持ち」フォーマットへのピボット

p274
- __`pivot`メソッド__ の逆を行う操作は __`pandas.melt`関数__

In [114]:
df = pd.DataFrame({
    'key': ['foo', 'bar', 'baz'],
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [115]:
# keyを非主キーにする
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [116]:
# key - variable 行列で値が value
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


p275
- インデックスのデータを行に戻して使いたい場合は、__`reset_index`メソッド__を用いるとよい。

In [117]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [118]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [119]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [120]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
